This notebook explores an alternative approach to produce the workbook map formulae (relationship between shpreadsheets).
It assumes `Workbook_map.output_formulae` and `Workbook_map.output_names` subroutine have been ran in the target workbook (eg. Workbook_map_EXAMPLE.xlsm) to produce a file with all the workbook's formulae (eg. Workbook_map_EXAMPLE_formulae.tsv) and all the names used (e.g. Workbook_map_EXAMPLE_names.tsv).

In [1]:
import pandas as pd
from pathlib import Path
import re

In [2]:
from itertools import chain
import collections

In [3]:
# TODO: Can we produce these files in python, without VBA?
formulae_sfp = Path('Workbook_map_EXAMPLE_formulae.tsv')
names_sfp = Path('Workbook_map_EXAMPLE_names.tsv')

In [4]:
formulae = pd.read_csv(formulae_sfp, sep='\t')
names = pd.read_csv(names_sfp, sep='\t')

In [5]:
# Ammend data to make it easier to find the sheet names
formulae['cell_formula_fixed'] = formulae['cell_formula'].str.strip("'=")
names['range'] = names['range'].str.strip("'=")

for tup in names.itertuples():
    name, rng = tup.name, tup.range
    formulae['cell_formula_fixed'] = formulae['cell_formula_fixed'].str.replace(name, rng)

In [6]:
formulae.head()

,sheet_name,cell_address,cell_formula,cell_formula_fixed
0,CALC_A_1,C3,'=INPUT1!C3,INPUT1!C3
1,CALC_A_1,C4,'=INPUT1!C4,INPUT1!C4
2,CALC_A_1,C5,'=INPUT1!C5,INPUT1!C5
3,CALC_A_1,C6,'=INPUT1!C6,INPUT1!C6
4,CALC_A_1,C7,'=INPUT1!C7,INPUT1!C7


In [7]:
names.head()

,name,range
0,_xlfn.STDEV.P,#NAME?
1,_xlfn.SUMIFS,#NAME?


In [8]:
# Find sheet names before an exclamation mark, either enclosed in single
# quotes, or not containing certain characters
sheetname_pat = r"""(?:'.+?'|\b[^:,/\*\-+ =&<>\[\]'"\(\)!]+?)(?=!)"""

In [9]:
sheetname_re = re.compile(sheetname_pat)

In [10]:
formulae.head()

,sheet_name,cell_address,cell_formula,cell_formula_fixed
0,CALC_A_1,C3,'=INPUT1!C3,INPUT1!C3
1,CALC_A_1,C4,'=INPUT1!C4,INPUT1!C4
2,CALC_A_1,C5,'=INPUT1!C5,INPUT1!C5
3,CALC_A_1,C6,'=INPUT1!C6,INPUT1!C6
4,CALC_A_1,C7,'=INPUT1!C7,INPUT1!C7


In [11]:
precedent_sheets = formulae['cell_formula_fixed'].map(sheetname_re.findall)

In [12]:
precedent_sheets.head()

0    [INPUT1]
1    [INPUT1]
2    [INPUT1]
3    [INPUT1]
4    [INPUT1]
Name: cell_formula_fixed, dtype: object

In [13]:
# Count the matched sheet names for each sheet
precedent_sheets_count = precedent_sheets.groupby(formulae['sheet_name']).apply(lambda S: collections.Counter(chain.from_iterable(S)))

In [14]:
precedent_sheets_count.index.names = ['sheet_name', 'sheet_name_precedent']
precedent_sheets_count = precedent_sheets_count.to_frame(name='count')

In [15]:
# Remove single quotes:
precedent_sheets_count = precedent_sheets_count.rename(lambda s: s.strip("'"), level=1)

In [16]:
# Remove sheet precedents to the same sheet (eg. a formula in SHEET1 like "=SHEET1 + 1")
filter_mask = (precedent_sheets_count.index.get_level_values('sheet_name') 
               != precedent_sheets_count.index.get_level_values('sheet_name_precedent'))
precedent_sheets_count = precedent_sheets_count[filter_mask]

In [17]:
precedent_sheets_count.head()

count
sheet_name sheet_name_precedent       
CALC_A_1   INPUT1                 10.0
CALC_A_2   INPUT1                 10.0
           INPUT2                 10.0
CALC_A_3   CALC_A_1               10.0
           CALC_A_2               10.0

In [18]:
# output
ofp = formulae_sfp.with_name(f'{formulae_sfp.stem}_count.csv')
precedent_sheets_count.to_csv(ofp)

In [19]:
import graphviz

In [20]:
# extra step to allow graphviz to be found 
import os
new_path = os.path.expanduser("~") + os.sep + r'AppData\Local\Continuum\anaconda3\Library\bin\graphviz'
os.environ["PATH"] += os.pathsep + new_path

In [21]:
g = graphviz.Digraph('G', filename=formulae_sfp.stem)

In [22]:
nodes = set(precedent_sheets_count.index.get_level_values('sheet_name'))
nodes.update(precedent_sheets_count.index.get_level_values('sheet_name_precedent'))

nodes = pd.Series(list(nodes))

In [23]:
# Crude classification
nodes.groupby(nodes.str.slice(0,1)).count()

C    8
I    4
R    2
S    2
dtype: int64

In [24]:
# Based on the classificaiton above
colour_dict = {
    'C': 'palegreen',
    'I': 'lightyellow',
    'O': 'lightblue',
    'R': 'lightblue',
}

In [25]:
# Define nodes first
for node in nodes:
    colour = colour_dict.get(node[0])
    
    kwargs = {'shape':'rectangle', 'style':'filled, rounded', 'color':'grey'}
    
    if colour:
        kwargs['color'] = colour
        
    g.attr('node', **kwargs)
    g.node(node)

In [26]:
# Define edges after nodes
for idx, rox in precedent_sheets_count.iterrows():
    sheet_name, precedent_name = idx
    g.edge(precedent_name, sheet_name)  # reverse, so inputs are on top

In [27]:
g.view()

'Workbook_map_EXAMPLE_formulae.pdf'